In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import emcee 
import pyimfit 
from astropy.io import fits
import corner
from IPython.display import Latex
import sys
import warnings
from matplotlib.ticker import LogFormatter
import pickle
import glob

from photutils.detection import find_peaks
from photutils.aperture import CircularAperture
from astropy.visualization import simple_norm
from astropy.stats import sigma_clipped_stats
from astropy.nddata import NDData
from photutils.psf import extract_stars
from photutils.psf import EPSFBuilder
from photutils import profiles
from astropy.table import Table

medium_font_size = 10
plt.rcParams['font.size'] = medium_font_size
plt.rcParams['axes.labelsize'] = medium_font_size
plt.rcParams['axes.titlesize'] = medium_font_size
plt.rcParams['xtick.labelsize'] = medium_font_size
plt.rcParams['ytick.labelsize'] = medium_font_size


plt.rcParams['font.family'] = 'monospace'
plt.rcParams['image.cmap'] = 'gray'
warnings.filterwarnings("ignore")

In [ ]:
def find_peaks_remove_dups(Imin):
    """Find peaks with pixel value > Imin
       remove duplicate values of x_peak
       find indices of detections that are too close/erred
       Return peak table with no duplicates and list of indices of close detections"""
    # FIND PEAKS
    peaks_tbl = find_peaks(data, threshold=Imin)   
    
    # Cleaning
    # search and remove sources with same x values, keeping only first finds
    df = peaks_tbl.to_pandas().sort_values(by='x_peak')
    dups = df.duplicated(subset="x_peak",keep="first")
    nodups = df[~dups]
    # list to remove sources with separation in x and y < 10 pixels 
    indices_to_drop = []
    for i in range(len(nodups)-1):
        if abs(nodups.iloc[i+1]['x_peak'] - nodups.iloc[i]['x_peak']) < 10 and abs(nodups.iloc[i+1]['y_peak'] - nodups.iloc[i]['y_peak']) < 10:
            indices_to_drop.append(nodups.iloc[i+1].name)
            indices_to_drop.append(nodups.iloc[i].name)

    indices_to_drop=list((set(indices_to_drop)))
    return nodups, indices_to_drop
    
    
def plot_erred_star_peaks(nodups,indices_to_drop,figfile):
    """plot duplicates detections that turns out to be erred stars"""
    nodups.loc[indices_to_drop].sort_values('x_peak')
    ncols = 6
    nrows = int(np.ceil(len(indices_to_drop)/ncols))
    fig,ax = plt.subplots(nrows,ncols,figsize=(ncols*2,nrows*2))
    ax = ax.ravel()
    for i in range(len(indices_to_drop)):
        y=int(nodups.loc[indices_to_drop[i]]['y_peak'])
        x=int(nodups.loc[indices_to_drop[i]]['x_peak'])
        px=30
        ax[i].imshow(data[y-px:y+px,x-px:x+px], origin='lower', cmap='viridis')
        ax[i].set_title(f"{x},{y}",fontsize=5)
    [ax[-i].axis('off') for i in range(1,len(ax)-len(indices_to_drop)+1)]
    fig.suptitle("Erred and duplicate detections of peaks",y=1.0)
    fig.tight_layout()
    fig.savefig(figfile)
    
    
def drop_erred_peaks(nodups, indices_to_drop):
    """drop erred indices from indice_to_drop from nodups,  returns table of sources position and pixel counts"""
    peaks_tbl = nodups.drop(indices_to_drop)
    return peaks_tbl


def plot_sources(peaks_tbl):
    """plot detected sources on the sky exposure"""
    positions = np.transpose((peaks_tbl['x_peak'], peaks_tbl['y_peak']))
    apertures = CircularAperture(positions, r=10.0)
    norm = simple_norm(data, 'log', percent=99.9)
    plt.figure(figsize=(12,12))
    plt.imshow(data, cmap='Greys_r', origin='lower', norm=norm,
               interpolation='nearest')
    apertures.plot(color='#0547f9', lw=0.5)
    plt.xlim(0, data.shape[1] - 1)
    plt.ylim(0, data.shape[0] - 1);
    
    
def plot_stars(stars):
    """plotting a table of star stamps"""
    nrows = int(np.ceil(len(stars)/4))
    ncols = 4
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(int(ncols*2), int(nrows*2)),
                           squeeze=True)
    ax = ax.ravel()
    for i in range(len(stars)):
        norm = simple_norm(stars[i], 'log', percent=99.0)
        im = ax[i].imshow(stars[i], norm=norm, origin='lower', cmap='viridis')
        ax[i].set_title(f"{i}, ({stars[i].center_flat[0][0]:.0f},{stars[i].center_flat[0][1]:.0f})")
        colorbar = fig.colorbar(im, ax=ax[i], shrink=0.8)
        colorbar.ax.yaxis.set_major_locator(plt.MaxNLocator(2))
    # turn off empty axes
    empty_axes = nrows*ncols-len(stars)
    [ax[-i].axis('off') for i in np.arange(1,empty_axes+1)]
    fig.tight_layout()
    fig.suptitle("Star ensemble for ePSF construction")
    
    
def make_star_cutout(peaks_tbl):
    """make 35x35 pixels star cutouts from peaks_tbl"""
    # size of a star cutout
    size = 35
    hsize = (size - 1) / 2
    x = peaks_tbl['x_peak']  
    y = peaks_tbl['y_peak']  
    # remove stars that are too close to edges
    mask = ((x > hsize) & (x < (data.shape[1] -1 - hsize)) &
            (y > hsize) & (y < (data.shape[0] -1 - hsize))) 
    stars_tbl = Table()
    stars_tbl['x'] = x[mask]  
    stars_tbl['y'] = y[mask]
    # extract stamps
    nddata = NDData(data=data)
    stars = extract_stars(nddata, stars_tbl, size=35)
    return stars_tbl, stars


def drop_star_stamps(stars_tbl,drop400,d400=True):
    stars_tbl.remove_rows(drop400)
    nddata = NDData(data=data)
    stars = extract_stars(nddata, stars_tbl, size=35)
    return stars


def build_psf(stars,osamp,shp,k='quartic',norm_r=20,maxit=30):
    """build epsf from star ensemble"""
    epsf_builder = EPSFBuilder(oversampling=osamp,shape=shp,smoothing_kernel=k,norm_radius=norm_r,maxiters=maxit) 
    epsf, fitted_stars = epsf_builder(stars) 
    return epsf,fitted_stars


def plot_psfs(epsf, epsf1, osamp):
    """plot to compare oversampled and not oversampled psfs"""
    fig,ax = plt.subplots(1,2)
    im0 = ax[0].imshow(epsf.data, cmap='viridis')
    im1= ax[1].imshow(epsf1.data, cmap='viridis')
    [fig.colorbar([im0,im1][i], ax=[ax[0],ax[1]][i],shrink=0.3) for i in range(2)]
    [ax[i].set_title([f'Oversampled PSF (x{osamp})', "PSF"][i]) for i in range(2)]
    fig.tight_layout();
    
def save_psf(epsf, filename):
    """save psf to fits file"""
    hdu = fits.PrimaryHDU(epsf.data)
    hdulist = fits.HDUList([hdu])
    hdulist.writeto(filename, overwrite=True)

In [ ]:
on = "J0912+0148"
sourceFile = glob.glob("/home/insepien/raw-data-agn/mos-fits-agn/2020_02_20_updated_20260105/*"+on+"*")[0]
data = fits.getdata(sourceFile)
# find peaks and remove duplicates, erred detections
nodups, indices_to_drop = find_peaks_remove_dups(400)
#---plot_erred_star_peaks(nodups,indices_to_drop)
# drop erred detections 
peaks_tbl = drop_erred_peaks(nodups, indices_to_drop)
# make star stamps
stars_tbl, stars = make_star_cutout(peaks_tbl)
# drop weird stars and make stamps again
d400 = "0 1 2 3 6 7 8 11 13 15 24 26 27 38 40 44 45 47 51 52 56 59 75 79 80 82 84 86 87 89 90 91 92 96 99 100".split(" ")
d400 = [int(i) for i in d400]
stars_clean = drop_star_stamps(stars_tbl,drop400=d400,d400=True)
# plot star stamps
#stars = drop_star_stamps(stars_tbl,drop400=[0,2,12,21,22,24],d400=True)
plot_stars(stars_clean)
# make psfs and plot
#epsf,fitted_stars = build_psf(stars,2,shp=None,k='quadratic')
# epsf1,fitted_stars1 = build_psf(stars,1,shp=(35,35),k='quartic')
# plot_psfs(epsf,epsf1,2)

### try star finders

In [ ]:
from photutils.detection import DAOStarFinder
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
import astropy.units as u

hdu = fits.getheader(sourceFile)
mean, median, std = sigma_clipped_stats(data, sigma=3.0)
### try finding star in only 1 quadrant
# find position of source in pix
pos = pd.read_csv("../cutouts/catalog.txt", delimiter=' ',header=None, names=['desig','ra','dec'])
wcs = WCS(hdu)
rapix, decpix = wcs.world_to_pixel(SkyCoord(pos[pos['desig'] == on].ra.values*u.deg, pos[pos['desig'] == on].dec.values*u.deg))
# mask everything but source quad
mask = np.zeros(data.shape, dtype=bool)
mask[:2200,:] = True
mask[2200:, 2200:] = True
# detect sources
daofind = DAOStarFinder(fwhm=hdu['FWHM_AVE'], threshold=5.*std)
sources = daofind(data,mask=mask)

In [ ]:
## plot to check source detection
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=4.0)
plt.imshow(data, norm='symlog',vmin=0,vmax=5)
plt.plot(rapix, decpix,"ro",markersize=50)
apertures.plot(color='blue', lw=1.5, alpha=0.5)
plt.colorbar();

In [ ]:
# extract star stamps
daopeak = sources.to_pandas()
daopeak.rename(columns={'xcentroid':'x_peak','ycentroid':'y_peak'},inplace=True)
daostars_tbl, daostars = make_star_cutout(daopeak[daopeak['peak']>400])
# clean the star tbl to remove duplicates
stars_sorted = daostars_tbl.to_pandas().copy()
stars_sorted.sort_values('x',inplace=True)
delx = np.diff(stars_sorted['x'])
dely = np.diff(stars_sorted['y'])
dupmask = (delx<30) & (dely<30)
selected_ind = list(stars_sorted.index.values[1:][~dupmask])
selected_ind.append(stars_sorted.index.values[0])
daostars_nodup = [daostars[i] for i in selected_ind]
# plot star no dup
plot_stars(daostars_nodup)

###############

In [ ]:
epsf,fitted_stars = build_psf(stars_clean,2,shp=None,k='quadratic')
epsf1,fitted_stars1 = build_psf(stars_clean,1,shp=(35,35),k='quartic')
plot_psfs(epsf,epsf1,2)

In [ ]:
fits.writeto("psf_j0912.fits", epsf1.data)

In [ ]:
nddata = NDData(data=data)
stb = Table()
stb['x'] =fitted_stars1.center_flat[:,0]
stb['y'] =fitted_stars1.center_flat[:,1]
recentered_stars = extract_stars(nddata, stb, size=35)
i=18
# fits.writeto("starPSF.fits", recentered_stars[i]/np.sum(recentered_stars[i]), overwrite=True)

### check radial profile consitency

In [ ]:
import os

def cal_percent_diff_psf_star_integrated_radial(filename):
    """find the percent differences of integrated radial profile"""
    d = pd.read_pickle("/home/insepien/research-data/psf-results/psf_pkls/"+filename)
    # normalize psf profile
    rp_psf1 = profiles.RadialProfile(d['psf'].data,xycen=d['psf'].origin,radii = np.arange(25))
    rp_psf1.normalize("max")
    # for each star
    percent_diff = []
    for i in range(len(d['fitted_stars'])):
        # normalize profile and integrate
        rp_star = profiles.RadialProfile(d['fitted_stars'][i].data,xycen=d['fitted_stars'][i].cutout_center,radii=np.arange(25))
        rp_star.normalize("max")
        star_sum = np.sum((-np.diff(rp_star.profile))*0.5)
        # get residual
        rp_resi = rp_star.profile - rp_psf1.profile
        # integrate residual
        resi_sum = np.sum((np.diff(rp_resi))*0.5)
        # find % diff
        percent_diff.append(resi_sum/star_sum*100)
    return percent_diff # return list, each element = percentage diff of resi/star RP integrated

filenames = os.listdir("/home/insepien/research-data/psf-results/psf_pkls")
pdiffs = []
for f in filenames: # loop for all targets
    if f[:3] == 'psf':
        pdiffs.append(cal_percent_diff_psf_star_integrated_radial(f))
    else: 
        print(f'err with file {f}')

pdiffs = pd.DataFrame(pdiffs)
plt.hist(pdiffs.values.flatten())
plt.xlabel("Int(resi)/Int(Avg star) * 100")

### RP plot and point source subtraction

In [ ]:
def plot_point_subtraction(epsf1,stars):
    """plot point source subtraction"""
    ncols = 5
    nrows = int(np.ceil(len(stars)/ncols))
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(int(ncols*2), int(nrows*2)),
                           squeeze=True)
    ax = ax.ravel()
    for n in range(len(stars)):
        resi = stars[n].compute_residual_image(epsf1)
        #err = np.sum(resi)/stars[n].estimate_flux()*100
        im = ax[n].imshow(resi)
        fig.colorbar(im,ax=ax[n],shrink=0.4)
        err = np.sum(resi)/stars[n].estimate_flux()*100
        ax[n].set_title(f"flux overshoot:{err:.2f}%")
    # turn off empty axes
    empty_axes = nrows*ncols-len(stars)
    [ax[-i].axis('off') for i in np.arange(1,empty_axes+1)]
    #err = np.sum(resi)/stars[n].estimate_flux()*100
    #fig.suptitle(f"average total flux overshoot:{err:.2f}%")
    fig.tight_layout();
    

def plot_sum_axis(epsf1, stars, Z, axisname):
    """plot stars and psf summed along an axis"""
    ncols = int(np.ceil(len(stars)/4))
    nrows = 4
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(int(ncols*3), int(nrows*2)),
                           squeeze=True)
    x,y =np.meshgrid(np.arange(35), np.arange(35))
    ax = ax.ravel()
    for i in range(len(stars)):
        psf = epsf1.evaluate(x=x,y=y,x_0=17,y_0=17,flux = stars[i].estimate_flux())
        ax[i].plot(np.sum(stars[i],axis=Z),"r")
        ax[i].plot(np.sum(psf,axis=Z),"b")
        ax[i].plot(np.sum(stars[i],axis=Z)-np.sum(psf,axis=Z),"k")
        ax[i].axvline(x=17,alpha=0.5)    
    # turn off empty axes
    empty_axes = nrows*ncols-len(stars)
    [ax[-i].axis('off') for i in np.arange(1,empty_axes+1)]
    fig.suptitle(f"Summing flux along {axisname}, blue-psf,red-star")
    fig.tight_layout();
    
    
def plot_psf_rProfile(epsf1,stars):
    """plot psf and stars radial profiles"""
    fig,ax = plt.subplots(1,2,figsize=(12,5))

    # plot x-sum in first axis
    for n in range(len(stars)):
        ax[0].plot(np.sum(stars[n].data, axis=0), "lightblue", alpha=0.5)
    ax[0].plot(np.sum(epsf1.data, axis=0), "steelblue")
    ax[0].axvline(x=stars[0].cutout_center[0],alpha=0.5)

    # plot radial profile in 2nd axis
    rp_psf1 = profiles.RadialProfile(epsf1.data,xycen=stars[0].cutout_center,radii = np.arange(25))
    rp_psf1.normalize("max")
    
    s = 0
    for i in range(len(stars)):
        rp_star = profiles.RadialProfile(stars[i].data,xycen=stars[i].cutout_center,radii=np.arange(25))
        rp_star.normalize("max")
        m = rp_star.area*rp_star.profile
        s+=np.sum(m[~np.isnan(m)])
        ax[1].plot(rp_star.radius, rp_star.profile, color="lightblue", alpha=0.5)
        ax[1].plot(rp_star.radius, rp_star.profile-rp_psf1.profile, "g", alpha=0.1)
    ax[1].plot(rp_psf1.radius, rp_psf1.profile,"steelblue")

    ax[0].set_title("x-axis profile")
    ax[0].set_xlabel("pixels")
    ax[0].set_ylabel("intensity(counts)")
    ax[1].set_title("Radial profile of stars and non-oversampled psf")
    ax[1].set_xlabel("pixels")
    ax[1].set_ylabel("intensity(counts)")
    legend_handles = [plt.Line2D([], [], color='lightblue', label='stars'),
                      plt.Line2D([], [], color='steelblue', label='PSF'),
                      plt.Line2D([], [], color='green', label='residual')]
    ax[1].legend(handles=legend_handles)
    psfsum = rp_psf1.area*rp_psf1.profile
    psf_integrated = np.sum(psfsum[~np.isnan(psfsum)])
    star_integrated = s/len(stars)
    ax[1].text(0.4, 0.5, f"Integrated PSF: {psf_integrated:.2f}", transform=ax[1].transAxes, fontsize=10, color='k')
    ax[1].text(0.4, 0.4, f"Ave. integrated stars: {star_integrated:.2f}", transform=ax[1].transAxes, fontsize=10, color='k')



In [ ]:
plot_psf_rProfile(epsf1,fitted_stars1)

In [ ]:
plot_point_subtraction(epsf1,fitted_stars1)

In [ ]:
starnum=0
psf1rp = profiles.RadialProfile(epsf1.data,xycen=stars[starnum].cutout_center,radii = np.arange(20))
starnrp = profiles.RadialProfile(stars[starnum].data,xycen=stars[starnum].cutout_center,radii = np.arange(20))
psf1rp.normalize("max")
starnrp.normalize("max")
fig,ax = plt.subplots(1,3,figsize=(10,4))
ax[0].plot(psf1rp.radius, psf1rp.profile,"steelblue")
ax[0].plot(starnrp.radius, starnrp.profile,"lightblue")
ax[0].plot(starnrp.radius, starnrp.profile-psf1rp.profile,"g", alpha=0.5)

ax[1].plot(np.sum(stars[starnum].data, axis=0), color="lightblue",marker="o")
shp = stars[starnum].shape[0]
cen = stars[starnum].cutout_center[0]
x,y =np.meshgrid(np.arange(shp), np.arange(shp))
psf = epsf1.evaluate(x=x,y=y,x_0=cen,y_0=cen,flux = stars[starnum].estimate_flux())
ax[1].plot(np.sum(psf, axis=0), "steelblue")
ax[1].plot(np.sum(psf, axis=0)-np.sum(stars[starnum].data, axis=0), "green", alpha=0.5)
ax[1].axvline(x=stars[starnum].cutout_center[0],c="b", alpha=0.2)

im2 = ax[2].imshow(-stars[starnum].compute_residual_image(epsf1))
fig.colorbar(im2, ax=ax[2], shrink=0.5)

fig = plt.figure()
legend_handles = [plt.Line2D([], [], color='lightblue', label='stars'),
                  plt.Line2D([], [], color='steelblue', label='PSF'),
                  plt.Line2D([], [], color='green', label='star-psf')]

[ax[i].set_title(['Radial profile','x-axis profile','PSF residual'][i]) for i in range(3)]

ax[0].legend(handles=legend_handles);


In [ ]:
sn = 2
fig,ax = plt.subplots(1,3,figsize=(12,4))
norm = simple_norm(stars[i], 'log', percent=99.0)
im0 = ax[0].imshow(stars[sn])
x,y =np.meshgrid(np.arange(35), np.arange(35))
psf = epsf1.evaluate(x=x,y=y,x_0=17,y_0=17,flux = stars[sn].estimate_flux())
im1 = ax[1].imshow(psf)
im2 = ax[2].imshow(psf-stars[sn].data)
[ax[i].plot(17,17,"ro", mfc='none') for i in range(2)] 
[fig.colorbar([im0,im1,im2][i], ax = ax[i],shrink=0.5) for i in range(3)];

In [ ]:
df = pd.DataFrame([[stars[i].origin, np.max(stars[i].data), stars[i].flux] for i in range(len(stars))],columns=['pos',"maxI","flux"])
fig,ax = plt.subplots(1,2,figsize=(8,3))
[ax[i].hist([df.maxI,df.flux][i],histtype="step") for i in range(2)]
[ax[i].set_xlabel(['Peak intensity of star(count)','Total star flux'][i]) for i in range(2)]
[ax[i].set_ylabel('Number of stars') for i in range(2)]
fig.tight_layout();